In [12]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
from time import sleep

from ICARUS.Database.Database_2D import Database_2D
from ICARUS.Database import XFLRDB

In [14]:
db = Database_2D()
db.load_data()

from ICARUS.Software.XFLR5.polars import read_polars_2d
read_polars_2d(db, XFLRDB)

airfoils = db.data

print(f"Airfoils that are computed are:")
for i , key in enumerate(list(airfoils.keys())):
    print(f"{i}: {key}")
sleep(1)
# index = int(input("Select an Airfoil: "))
index = 8
airfoil = list(airfoils.keys())[index]

Airfoils that are computed are:
0: NACA0008
1: NACA0012
2: NACA0015
3: NACA2412
4: NACA4413
5: NACA4415
6: NACA4419
7: NACA4438
8: NACA64418_flap_35o_0.75h_1.3c
9: NACA64418


In [15]:
from ICARUS.Core.Units import calc_reynolds
import numpy as np
from numpy import ndarray, floating, dtype
from typing import Any
# PARAMETERS FOR ESTIMATION
chord_max: float = 5.6
chord_min: float = 2.3
u_max: float = 140
u_min: float = 100
viscosity: float = 1.56e-5

# MACH ESTIMATION
mach_max: float = 0.01
# mach_min: float = calc_mach(10, speed_of_sound)
# mach: ndarray[Any, dtype[floating[Any]]] = np.linspace(mach_max, mach_min, 10)
MACH: float = mach_max

# REYNOLDS ESTIMATION
reynolds_max: float = calc_reynolds(u_max, chord_max, viscosity)
reynolds_min: float = calc_reynolds(u_min, chord_min, viscosity)
reynolds: ndarray[Any, dtype[floating[Any]]] = np.logspace(
    start=np.log10(reynolds_min),
    stop=np.log10(reynolds_max),
    num=5,
    base=10,
)

# ANGLE OF ATTACK SETUP
aoa_min: float = -6
aoa_max: float = 6
num_of_angles: int = int((aoa_max - aoa_min) * 2 + 1)
angles: ndarray[Any, dtype[floating[Any]]] = np.linspace(
    start=aoa_min,
    stop=aoa_max,
    num=num_of_angles,
)

# Transition to turbulent Boundary Layer
ftrip_up: dict[str, float] = {"pos": 0.2, "neg": 0.1}
ftrip_low: dict[str, float] = {"pos": 0.1, "neg": 0.2}
Ncrit = 9

In [20]:
from ICARUS.Airfoils.airfoilD import AirfoilD


naca64418: AirfoilD = db.set_available_airfoils()["NACA64418"]
naca64418.flap_airfoil(0.7, 1.3, 35)
# plt.title("NACA64418 Flapped")

In [21]:
naca64418.save_le(XFLRDB)

Saving airfoil to /home/tryfonas/data/Uni/Software/hermes/Data/XFLR5/64418_flap_35o_0.7h_1.3c.dat


In [9]:
from ICARUS.Software.Xfoil.analyses.angles import single_reynolds_run, single_reynolds_run_seq

a = single_reynolds_run_seq(
    Reyn = reynolds[-1],
    MACH = MACH,
    angles=angles,
    airfoil = naca64418,
    solver_options= {
    "Ncrit" : Ncrit,
    "ftrip_low" : ftrip_low,
    "ftrip_up" : ftrip_up,
    "print": False
    }
)
# numpy remove nan
a = a[~np.isnan(a).any(axis=1)]

In [11]:
import matplotlib.pyplot as plt
import scipy

# Make 4 subplots for the 4 different plots
fig, axs = plt.subplots(2, 2)

# Plot the Cl vs AoA
axs[0, 0].plot(a[:, 0], a[:, 1])
# # Make a linear interpolation to extend the curve of the Cl vs AoA to 5 degrees dotted
# # fit a curve to the data points using a 3rd order polynomial
# z = np.polyfit(a[:, 0], a[:, 1], 3)
# # construct the polynomial
# p = np.poly1d(z)
# # construct the x values for the linear interpolation
# x = np.linspace(a[:, -1][0], 5, 100)
# # construct the y values for the linear interpolation
# y = p(x)
# # plot the linear interpolation
# axs[0, 0].plot(x, y, linestyle="--", color="black")

axs[0, 0].set_title("Cl vs AoA")
axs[0, 0].set_xlabel("AoA [deg]")
axs[0, 0].set_ylabel("Cl [-]")
axs[0, 0].grid()

# Plot the Cd vs AoA
axs[0, 1].plot(a[:, 0], a[:, 2])
axs[0, 1].set_title("Cd vs AoA")
axs[0, 1].set_xlabel("AoA [deg]")
axs[0, 1].set_ylabel("Cd [-]")
axs[0, 1].grid()

# Plot the Cm vs AoA
axs[1, 0].plot(a[:, 0], a[:, 3])
axs[1, 0].set_title("Cm vs AoA")
axs[1, 0].set_xlabel("AoA [deg]")
axs[1, 0].set_ylabel("Cm [-]")
axs[1, 0].grid()

# Plot the Cl/Cd vs AoA
axs[1, 1].plot(a[:, 0], a[:, 1]/ a[:, 2])
axs[1, 1].set_title("Cl/Cd vs AoA")
axs[1, 1].set_xlabel("AoA [deg]")
axs[1, 1].set_ylabel("Cl/Cd [-]")
axs[1, 1].grid()

# Make the plots look nice
fig.tight_layout()

# Show the plots
plt.show()
